In [16]:
# Animated train movement with speed labels and explicit direction control

import math
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.patches import Rectangle, Polygon, Circle

# Scenario: List of events - ('direction', 'FRONT'|'REAR') or ('speed', kmh, seconds)
scenario_events = [
    ('direction', 'FRONT'),
    ('speed', 5, 10),
    ('speed', 10, 10),
    ('speed', 15, 10),
    ('speed', 20, 60),  # max-speed endurance
    ('speed', 15, 10),
    ('speed', 10, 10),
    ('speed', 5, 10),
    ('speed', 0, 20),   # stop
    ('direction', 'REAR'),
    ('speed', 5, 10),
    ('speed', 10, 10),
    ('speed', 15, 10),
    ('speed', 20, 60),  # max-speed endurance
    ('speed', 10, 10),
    ('speed', 5, 10),
    ('speed', 0, 20),   # stop
]

# Build per-second series: time, speed (km/h), position (axis units)
track_length = 200.0
scale_units_per_mps = 0.2  # tune to control how far the train moves for a given speed

time = [0]
speed_kmh = [0.0]
pos = [5.0]  # start near the left

direction = 1  # +1: FRONT (right), -1: REAR (left)

t = 0
x = pos[0]

for event in scenario_events:
    if event[0] == 'direction':
        # Set direction: 'FRONT' = +1 (right), 'REAR' = -1 (left)
        direction = +1 if event[1] == 'FRONT' else -1
    elif event[0] == 'speed':
        s_kmh, duration_s = event[1], event[2]
        v_mps = s_kmh * 1000.0 / 3600.0
        for _ in range(int(duration_s)):
            t += 1
            time.append(t)
            speed_kmh.append(float(s_kmh))
            
            # Move according to current speed and direction
            x += direction * v_mps * scale_units_per_mps
            
            # Keep within track bounds
            if x < 0:
                x = 0
            elif x > track_length:
                x = track_length
            
            pos.append(x)

# Set up animation figure
fig, ax = plt.subplots(figsize=(8, 3.2))
ax.set_xlim(0, track_length)
ax.set_ylim(-5, 5)
ax.axis('off')

# Draw track
ax.plot([0, track_length], [0, 0], color='#888', linewidth=3)
for m in range(0, int(track_length) + 1, 10):
    ax.plot([m, m], [-0.8, 0.8], color='#bbb', linewidth=1)

# Train geometry
train_width = 8.0
train_height = 2.2
wheel_r = 0.3

# Main body
train = Rectangle((pos[0] - train_width / 2, -train_height / 2), train_width, train_height,
                  facecolor='#e0e0e0', edgecolor='#424242', linewidth=1.5)
ax.add_patch(train)

# Nose (simple wedge)
nose_offsets = [
    (train_width/2, -train_height/2),
    (train_width/2 + 1.4, 0.0),
    (train_width/2,  train_height/2)
]
nose = Polygon([[pos[0] + dx, dy] for dx, dy in nose_offsets], closed=True,
                facecolor='#bdbdbd', edgecolor='#616161', linewidth=1.2)
ax.add_patch(nose)

# Speed stripe (color-coded)
stripe_height = 0.5
stripe = Rectangle((pos[0] - train_width/2 + 0.4, 0.2), train_width - 0.8, stripe_height,
                    facecolor='#2e7d32', edgecolor='none')
ax.add_patch(stripe)

# Windows
window_w, window_h = 1.0, 0.6
window_offsets = [1.0, 3.0, 5.0]
windows = [
    Rectangle((pos[0] - train_width/2 + off, 0.9), window_w, window_h,
              facecolor='#bbdefb', edgecolor='#90caf9', linewidth=1.0)
    for off in window_offsets
 ]
for w in windows:
    ax.add_patch(w)

# Wheels
wheel_offsets = [-train_width*0.25, train_width*0.25]
wheels = [Circle((pos[0] + off, -1.3), radius=wheel_r, facecolor='#424242', edgecolor='#212121')
          for off in wheel_offsets]
for w in wheels:
    ax.add_patch(w)

# Headlight
headlight = Circle((pos[0] + train_width/2 + 1.0, 0.0), radius=0.22, facecolor='#ffe082', edgecolor='#ffd54f')
ax.add_patch(headlight)

# Speed and direction label near the train
label = ax.text(pos[0], 3.0, '', ha='center', va='bottom', fontsize=11, color='#333')
# Global title/info (time only)
info = ax.text(track_length / 2, 4.4, '', ha='center', va='bottom', fontsize=12, color='#111')

# Helper to get direction arrow for each frame by comparing positions
def direction_arrow(i):
    if i == 0:
        return '→'
    dx = pos[i] - pos[i - 1]
    if dx > 1e-6:
        return '→'
    elif dx < -1e-6:
        return '←'
    else:
        return '•'

# Helper to update train x-position
def set_train_x(x):
    # Body
    train.set_x(x - train_width / 2)
    # Stripe
    stripe.set_x(x - train_width/2 + 0.4)
    # Windows
    for patch, off in zip(windows, window_offsets):
        patch.set_x(x - train_width/2 + off)
    # Wheels
    for patch, off in zip(wheels, wheel_offsets):
        patch.center = (x + off, -1.3)
    # Nose
    nose.set_xy([[x + dx, dy] for dx, dy in nose_offsets])
    # Headlight
    headlight.center = (x + train_width/2 + 1.0, 0.0)

# Update function
def update(frame):
    x = pos[frame]
    v = speed_kmh[frame]
    arr = direction_arrow(frame)

    set_train_x(x)
    label.set_position((x, 3.0))
    label.set_text(f"{arr}  {v:.0f} km/h")

    # Color-code speed (slow=green, medium=orange, fast=red)
    if v <= 10:
        speed_color = '#2e7d32'
    elif v <= 15:
        speed_color = '#f9a825'
    else:
        speed_color = '#c62828'
    stripe.set_facecolor(speed_color)
    headlight.set_facecolor('#ffd54f' if v > 0 else '#ffe082')

    # Time only (position removed)
    info.set_text(f"Time: {time[frame]} s")
    return (train, nose, stripe, *windows, *wheels, headlight, label, info)

# Animate
ani = FuncAnimation(fig, update, frames=len(time), interval=50, blit=False)

# Save GIF
output_path = "./train_movement.gif"
writer = PillowWriter(fps=20)
ani.save(output_path, writer=writer, dpi=100)

print(output_path)

plt.close(fig)

./train_movement.gif
